##### Copyright 2023 Google LLC.

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini API: Document Q&A with ChromaDB

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/examples/chromadb/Vectordb_with_chroma.ipynb"><img src="https://github.com/google-gemini/cookbook/blob/main/images/colab_logo_32px.png?raw=1" />Run in Google Colab</a>
  </td>
</table>


## Overview

This tutorial demonstrates how to use the Gemini API to create a vector database and retrieve answers to questions from the database. Moreover, you will use [ChromaDB](https://docs.trychroma.com/){:.external}, an open-source Python tool that creates embedding databases. ChromaDB allows you to:

* Store embeddings as well as their metadata
* Embed documents and queries
* Search through the database of embeddings

In this tutorial, you'll use embeddings to retrieve an answer from a database of vectors created with ChromaDB.

## Setup

First, download and install ChromaDB and the Gemini API Python library.

In [33]:
!pip install -U -q "google-genai"
!pip install -q chromadb

Then import the modules you'll use in this tutorial.

In [34]:
import textwrap
import chromadb
import numpy as np
import pandas as pd

from google import genai

from IPython.display import Markdown
from chromadb import Documents, EmbeddingFunction, Embeddings

## Configure your API key

To run the following cell, your API key must be stored in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see [Authentication](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Authentication.ipynb) for an example.


In [35]:
from google.colab import userdata
api_key = userdata.get('GOOGLE_API_KEY')

client=genai.Client(api_key=api_key)

Key Point: Next, you will choose a model. Any embedding model will work for this tutorial, but for real applications it's important to choose a specific model and stick with it. The outputs of different models are not compatible with each other.
List of Embedding Models Available:

**Note**: At this time, the Gemini API is [only available in certain regions](https://ai.google.dev/available_regions).

In [36]:
for model in client.models.list():
  if 'embedContent' in model.supported_actions:
    print(model.name)

models/embedding-001
models/text-embedding-004


### Data

Here is a small set of documents you will use to create an embedding database:

In [37]:
DOCUMENT1 = "Operating the Climate Control System  Your Googlecar has a climate control system that allows you to adjust the temperature and airflow in the car. To operate the climate control system, use the buttons and knobs located on the center console.  Temperature: The temperature knob controls the temperature inside the car. Turn the knob clockwise to increase the temperature or counterclockwise to decrease the temperature. Airflow: The airflow knob controls the amount of airflow inside the car. Turn the knob clockwise to increase the airflow or counterclockwise to decrease the airflow. Fan speed: The fan speed knob controls the speed of the fan. Turn the knob clockwise to increase the fan speed or counterclockwise to decrease the fan speed. Mode: The mode button allows you to select the desired mode. The available modes are: Auto: The car will automatically adjust the temperature and airflow to maintain a comfortable level. Cool: The car will blow cool air into the car. Heat: The car will blow warm air into the car. Defrost: The car will blow warm air onto the windshield to defrost it."
DOCUMENT2 = "Your Googlecar has a large touchscreen display that provides access to a variety of features, including navigation, entertainment, and climate control. To use the touchscreen display, simply touch the desired icon.  For example, you can touch the \"Navigation\" icon to get directions to your destination or touch the \"Music\" icon to play your favorite songs."
DOCUMENT3 = "Shifting Gears Your Googlecar has an automatic transmission. To shift gears, simply move the shift lever to the desired position.  Park: This position is used when you are parked. The wheels are locked and the car cannot move. Reverse: This position is used to back up. Neutral: This position is used when you are stopped at a light or in traffic. The car is not in gear and will not move unless you press the gas pedal. Drive: This position is used to drive forward. Low: This position is used for driving in snow or other slippery conditions."

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

## Creating the embedding database with ChromaDB

You will create a [custom function](https://docs.trychroma.com/embeddings#custom-embedding-functions){:.external} for performing embedding using the Gemini API. By inputting a set of documents into this custom function, you will receive vectors, or embeddings of the documents.


### API changes to Embeddings with model embedding-001

For the new embeddings model, embedding-001, there is a new task type parameter and the optional title (only valid with task_type=`RETRIEVAL_DOCUMENT`).

These new parameters apply only to the newest embeddings models.The task types are:

Task Type | Description
---       | ---
RETRIEVAL_QUERY	| Specifies the given text is a query in a search/retrieval setting.
RETRIEVAL_DOCUMENT | Specifies the given text is a document in a search/retrieval setting.
SEMANTIC_SIMILARITY	| Specifies the given text will be used for Semantic Textual Similarity (STS).
CLASSIFICATION	| Specifies that the embeddings will be used for classification.
CLUSTERING	| Specifies that the embeddings will be used for clustering.

In [30]:
from google.genai.types import EmbedContentConfig
class GeminiEmbeddingFunction(EmbeddingFunction):
  def __call__(self, input: Documents) -> Embeddings:
    model = 'models/embedding-001'
    return client.models.embed_content(model=model,contents=Documents,config=EmbedContentConfig(task_type='RETRIEVAL_DOCUMENT'))


Now you will create the vector database. In the `create_chroma_db` function, you will instantiate a [Chroma client](https://docs.trychroma.com/getting-started){:.external}. From there, you will create a collection, which is where you store your embeddings, documents, and any metadata. Note that the embedding function from above is passed as an argument to the `create_collection`.

Next, you use the `add` method to add the documents to the collection.

In [31]:
def create_chroma_db(documents, name):
  chroma_client = chromadb.Client()
  try:
    db=chroma_client.get_collection(name=name)
    message="Collection already exists"
  except:
    db = chroma_client.create_collection(name=name, embedding_function=GeminiEmbeddingFunction())
    message="Collection created"

  for i, d in enumerate(documents):
    db.add(
      documents=d,
      ids=str(i)
    )
  return (db,message)

In [32]:
# Set up the DB if found
db,message = create_chroma_db(documents, "googlecarsdatabase")
print(message)

/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:01<00:00, 44.2MiB/s]


Collection already exists


Confirm that the data was inserted by looking at the database:

In [38]:
import pandas as pd
from IPython.display import display

peeked_data = db.peek(3)
embeddings_list = [peeked_data['embeddings'][int(id)] for id in peeked_data['ids']] # Cast id to int here

df=pd.DataFrame({
    'ids':peeked_data['ids'],
    'embeddings':embeddings_list,
    'documents':peeked_data['documents'],
    'metadatas':peeked_data['metadatas'],
    'uris':peeked_data['uris'],
    'data':peeked_data['data'],
    'included':peeked_data['included']
       })
display(df)

,ids,embeddings,documents,metadatas,uris,data,included
0,0,"[-0.01275272574275732, 0.018082624301314354, 0...",Operating the Climate Control System Your Goo...,None,None,None,IncludeEnum.embeddings
1,1,"[0.0002497922978363931, 0.0011292541166767478,...",Your Googlecar has a large touchscreen display...,None,None,None,IncludeEnum.documents
2,2,"[-0.058870501816272736, -0.04798182100057602, ...",Shifting Gears Your Googlecar has an automatic...,None,None,None,IncludeEnum.metadatas


## Getting the relevant document

`db` is a Chroma collection object. You can call `query` on it to perform a nearest neighbors search to find similar embeddings or documents.


In [39]:
def get_relevant_passage(query, db):
  passage = db.query(query_texts=[query], n_results=1)['documents'][0][0]
  return passage

In [40]:
# Perform embedding search
passage = get_relevant_passage("touch screen features", db)
Markdown(passage)

Your Googlecar has a large touchscreen display that provides access to a variety of features, including navigation, entertainment, and climate control. To use the touchscreen display, simply touch the desired icon.  For example, you can touch the "Navigation" icon to get directions to your destination or touch the "Music" icon to play your favorite songs.

Now that you have found the relevant passage in your set of documents, you can use it make a prompt to pass into the Gemini API.

In [41]:
def make_prompt(query, relevant_passage):
  escaped = relevant_passage.replace("'", "").replace('"', "").replace("\n", " ")
  prompt = ("""You are a helpful and informative bot that answers questions using text from the reference passage included below. \
  Be sure to respond in a complete sentence, being comprehensive, including all relevant background information. \
  However, you are talking to a non-technical audience, so be sure to break down complicated concepts and \
  strike a friendly and converstional tone. \
  If the passage is irrelevant to the answer, you may ignore it.
  QUESTION: '{query}'
  PASSAGE: '{relevant_passage}'

    ANSWER:
  """).format(query=query, relevant_passage=escaped)

  return prompt

Pass a query to the prompt:

In [42]:
query = "How do you use the touchscreen in the Google car?"
prompt = make_prompt(query, passage)
Markdown(prompt)

You are a helpful and informative bot that answers questions using text from the reference passage included below.   Be sure to respond in a complete sentence, being comprehensive, including all relevant background information.   However, you are talking to a non-technical audience, so be sure to break down complicated concepts and   strike a friendly and converstional tone.   If the passage is irrelevant to the answer, you may ignore it.
  QUESTION: 'How do you use the touchscreen in the Google car?'
  PASSAGE: 'Your Googlecar has a large touchscreen display that provides access to a variety of features, including navigation, entertainment, and climate control. To use the touchscreen display, simply touch the desired icon.  For example, you can touch the Navigation icon to get directions to your destination or touch the Music icon to play your favorite songs.'

    ANSWER:
  

Now use the `generate_content` method to to generate a response from the model.

In [43]:
model = 'gemini-2.0-flash'
answer = client.models.generate_content(
    model=model,
    contents=prompt
)
Markdown(answer.text)

Okay, so your Googlecar comes with a big touchscreen that lets you do all sorts of things! You can use it to get where you're going by touching the Navigation icon, or you can listen to music by touching the Music icon. Basically, if you want to use a feature, just touch the icon for it on the screen. It's just like using a big smartphone or tablet!


## Next steps

To learn more about how you can use the embeddings, check out the [examples](https://ai.google.dev/examples?keywords=embed) available. To learn how to use other services in the Gemini API, visit the [Python quickstart](https://ai.google.dev/gemini-api/docs/get-started/python).